In [1]:
import pandas as pd
import requests, re
from bs4 import BeautifulSoup

In [2]:
url = 'http://www.statssa.gov.za/?page_id=993&id=city-of-johannesburg-municipality' # base url
res = requests.get(url) # request
soup = BeautifulSoup(res.content, 'lxml') # beautiful soup

In [3]:
lir = soup.find_all('li') # list of the cities

In [4]:
city = pd.read_csv('Data/localities.csv') # read csv
city = list(city['City']) # list of cities

In [5]:
cities = [] # empty list
for i in lir:  # extracting url structure
    dict = {}
    if len(re.findall('(?<=a href=")\?\w+\W+\d+\W+\w+\W+\w+\W+\d+(?=">)',str(i.contents))) > 0:
        dict['href'] = re.findall('(?<=a href=")\?\w+\W+\d+\W+\w+\W+\w+\W+\d+(?=">)',str(i.contents))
        cities.append(dict)
    else:
        None

In [6]:
for i in range(len(cities)): # cleaning up url
    cities[i-1]['href'][0] = cities[i-1]['href'][0].replace('amp;', '')

In [7]:
# example page 'http://www.statssa.gov.za/?page_id=4286&id=11305'

click_url = {} # list of cleaned urls
for i in range(len(cities)):
    click_url[i] = f"http://www.statssa.gov.za/{cities[i]['href'][0]}"

In [8]:
city_master = [] # empty list of city data
    
for y in range(len(city)-1): # loop through all cities

    citi = []

    res = requests.get(click_url[y])
    soup = BeautifulSoup(res.content, 'lxml')
    tdr = soup.find_all('td')

    stats = []

    for x in tdr: # going through all the elements
        dict1 = {}
        dict1['element'] = x.text
        stats.append(dict1)

    for i in range(len(stats)-1):

        dict2 = {}

        if stats[i]['element'] == 'Total population':  # filtering data
            dict2['Total population'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Young (0-14)':
            dict2['Young (0-14)'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Working Age (15-64)':
            dict2['Working Age (15-64)'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Elderly (65+)':
            dict2['Elderly (65+)'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Dependency ratio':
            dict2['Dependency ratio'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Sex ratio':
            dict2['Sex ratio'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Population density':
            dict2['Population density'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'No schooling aged 20+':
            dict2['No schooling aged 20+'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Higher education aged 20+':
            dict2['Higher education aged 20+'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Matric aged 20+':
            dict2['Matric aged 20+'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Number of households':
            dict2['Number of households'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Average household size':
            dict2['Average household size'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Female headed households':
            dict2['Female headed households'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Housing owned/paying off':
            dict2['Housing owned/paying off'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Flush toilet connected to sewerage':
            dict2['Flush toilet connected to sewerage'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Weekly refuse removal':
            dict2['Weekly refuse removal'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Piped water inside dwelling':
            dict2['Piped water inside dwelling'] = stats[i+1]['element']
            citi.append(dict2)
        if stats[i]['element'] == 'Electricity for lighting':
            dict2['Electricity for lighting'] = stats[i+1]['element']
            citi.append(dict2)

        dict = {city[y] : citi} # adding data to dictionary
    city_master.append(dict) # append to final dictionary

In [10]:
columns = list(pd.DataFrame(city_master[0][city[0]]).columns)

In [81]:
columns

['Total population',
 'Young (0-14)',
 'Working Age (15-64)',
 'Elderly (65+)',
 'Dependency ratio',
 'Sex ratio',
 'Population density',
 'No schooling aged 20+',
 'Higher education aged 20+',
 'Matric aged 20+',
 'Number of households',
 'Average household size',
 'Female headed households',
 'Housing owned/paying off',
 'Flush toilet connected to sewerage',
 'Weekly refuse removal',
 'Piped water inside dwelling',
 'Electricity for lighting']

In [192]:
df

,city
0,Alexandra
1,Chartwell
2,City of Johannesburg NU
3,Dainfern
4,Diepsloot
5,Drie Ziek
6,Ebony Park
7,Ennerdale
8,Farmall
9,Itsoseng


In [71]:
df = []
for i in range(len(city_master)-1):
    dict = {}
    dict['city'] = list(city_master[i])[0]
    df.append(dict)
df = pd.DataFrame(df)

In [103]:
df2 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['total population'] = city_master[num][city[num]][0]['Total population']
    df2.append(dict)
df2 = pd.DataFrame(df2)

In [161]:
df3 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Young (0-14)'] = city_master[num][city[num]][1]['Young (0-14)']
    df3.append(dict)
df3 = pd.DataFrame(df3)

In [154]:
df4 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Working Age (15-64)'] = city_master[num][city[num]][2]['Working Age (15-64)']
    df4.append(dict)
df4 = pd.DataFrame(df4)

In [157]:
df5 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Elderly (65+)'] = city_master[num][city[num]][3]['Elderly (65+)']
    df5.append(dict)
df5 = pd.DataFrame(df5)

In [159]:
df6 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Dependency ratio'] = city_master[num][city[num]][4]['Dependency ratio']
    df6.append(dict)
df6 = pd.DataFrame(df6)

In [163]:
df7 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Sex ratio'] = city_master[num][city[num]][5]['Sex ratio']
    df7.append(dict)
df7 = pd.DataFrame(df7)

In [166]:
df8 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Population density'] = city_master[num][city[num]][6]['Population density']
    df8.append(dict)
df8 = pd.DataFrame(df8)

In [168]:
df9 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['No schooling aged 20+'] = city_master[num][city[num]][7]['No schooling aged 20+']
    df9.append(dict)
df9 = pd.DataFrame(df9)

In [170]:
df10 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Higher education aged 20+'] = city_master[num][city[num]][8]['Higher education aged 20+']
    df10.append(dict)
df10 = pd.DataFrame(df10)

In [172]:
df11 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Matric aged 20+'] = city_master[num][city[num]][9]['Matric aged 20+']
    df11.append(dict)
df11 = pd.DataFrame(df11)

In [175]:
df12 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Number of households'] = city_master[num][city[num]][10]['Number of households']
    df12.append(dict)
df12 = pd.DataFrame(df12)

In [177]:
df13 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Average household size'] = city_master[num][city[num]][11]['Average household size']
    df13.append(dict)
df13 = pd.DataFrame(df13)

In [179]:
df14 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Female headed households'] = city_master[num][city[num]][12]['Female headed households']
    df14.append(dict)
df14 = pd.DataFrame(df14)

In [181]:
df15 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Housing owned/paying off'] = city_master[num][city[num]][13]['Housing owned/paying off']
    df15.append(dict)
df15 = pd.DataFrame(df15)

In [183]:
df16 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Flush toilet connected to sewerage'] = city_master[num][city[num]][14]['Flush toilet connected to sewerage']
    df16.append(dict)
df16 = pd.DataFrame(df16)

In [185]:
df17 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Weekly refuse removal'] = city_master[num][city[num]][15]['Weekly refuse removal']
    df17.append(dict)
df17 = pd.DataFrame(df17)

In [187]:
df18 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Piped water inside dwelling'] = city_master[num][city[num]][16]['Piped water inside dwelling']
    df18.append(dict)
df18 = pd.DataFrame(df18)

In [189]:
df19 = []
for num in range(len(city_master)-1):
    dict = {}
    dict['Electricity for lighting'] = city_master[num][city[num]][17]['Electricity for lighting']
    df19.append(dict)
df18 = pd.DataFrame(df19)

In [195]:
df_final = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18], axis = 1)

In [199]:
df_final.to_csv('output.csv', index = False)